In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
cd drive


/content/drive


In [3]:
cd My Drive

/content/drive/My Drive


In [4]:
cd DL2

/content/drive/My Drive/DL2


In [0]:
import torch   

#handling text data
from torchtext import data   

In [0]:
# SEED = 2019

#Torch
# torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

In [0]:
# import csv
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# import string
# import pandas as pd
# import numpy as np

# def preprocessing(text):
#   # split into words
#   tokens = text.split()
#   # tokens = word_tokenize(text)
#   # convert to lower case
#   words = [w for w in tokens if w[0] != '@' ]
#   # print(words)
#   tokens = [w.lower() for w in words]
#   # remove punctuation from each word

#   table = str.maketrans('', '', string.punctuation)
#   stripped = [w.translate(table) for w in tokens]
#   # remove remaining tokens that are not alphabetic
#   words = [word for word in stripped if word.isalpha()]
#   # filter out stop words

#   # stop_words = set(stopwords.words('english'))
  
#   st = ""
#   for w in words:
#     # print(w)
#     if w[0] != '@':
#       st = st + w + " "
#     # print(st)
#   return st
# # print(words[:100])
# # preprocessing("I am  A GOOOD.... Girls @Sas !!!")
# data = pd.read_csv('train_data.csv')
# data = np.array(data)
# labels = data[1 :, 0]
# text = data[1:, 1]
# preprocessed_text = []
# for i in range(len(text)):
#   preprocessed_text.append(preprocessing(text[i]))
# data_tuples = list(zip(labels, preprocessed_text))
# df = pd.DataFrame(data_tuples, columns=['Labels','Text'])
# df.to_csv(r'preprocessed_train_dataf.csv')

In [8]:

tokenize = lambda x: x.split()
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
LABEL = data.LabelField()
fields = [(None, None), ('label', LABEL), ('text',TEXT)]
# fields = [('label', LABEL), ('text',TEXT)]
train_data = data.TabularDataset(path = 'p_training_data.csv',fields = fields, format = 'csv',skip_header = True)
valid_data = data.TabularDataset(path = 'p_validation_data.csv',fields = fields, format = 'csv',skip_header = True)
#print preprocessed text
print(vars(train_data.examples[0]))

{'label': 'sadness', 'text': ['layin', 'n', 'bed', 'with', 'a', 'headache', 'ughhhhwaitin', 'on', 'your', 'call']}


In [0]:
# import random
# train_data, valid_data = training_data.split(split_ratio=0.8, random_state = random.seed(SEED))

In [10]:
vars(train_data[0])

{'label': 'sadness',
 'text': ['layin',
  'n',
  'bed',
  'with',
  'a',
  'headache',
  'ughhhhwaitin',
  'on',
  'your',
  'call']}

In [11]:
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.840B.300d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)  
word_embeddings = TEXT.vocab.vectors 
vocab_size = len(TEXT.vocab)

Size of TEXT vocabulary: 6188
Size of LABEL vocabulary: 13
[('i', 13324), ('to', 9146), ('the', 7597), ('a', 5671), ('my', 5352), ('and', 4400), ('it', 3653), ('is', 3602), ('you', 3464), ('in', 3409)]
defaultdict(<function _default_unk_index at 0x7f67cb34b6a8>, {'<unk>': 0, '<pad>': 1, 'i': 2, 'to': 3, 'the': 4, 'a': 5, 'my': 6, 'and': 7, 'it': 8, 'is': 9, 'you': 10, 'in': 11, 'for': 12, 'im': 13, 'of': 14, 'me': 15, 'on': 16, 'have': 17, 'so': 18, 'but': 19, 'that': 20, 'not': 21, 'just': 22, 'its': 23, 'be': 24, 'at': 25, 'was': 26, 'with': 27, 'no': 28, 'this': 29, 'now': 30, 'get': 31, 'up': 32, 'out': 33, 'work': 34, 'go': 35, 'dont': 36, 'like': 37, 'cant': 38, 'all': 39, 'are': 40, 'day': 41, 'do': 42, 'got': 43, 'good': 44, 'today': 45, 'too': 46, 'going': 47, 'really': 48, 'one': 49, 'back': 50, 'know': 51, 'am': 52, 'from': 53, 'what': 54, 'your': 55, 'want': 56, 'time': 57, 'miss': 58, 'about': 59, 'will': 60, 'we': 61, 'still': 62, 'lol': 63, 'home': 64, 'u': 65, 'see': 66

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 32

#Load an iterator
train_iter, valid_iter = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

class RNN(nn.Module):
  def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
    super(RNN, self).__init__()

    """
    Arguments
    ---------
    batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
    output_size : 13 
    hidden_sie : Size of the hidden_state of the LSTM
    vocab_size : Size of the vocabulary containing unique words
    embedding_length : Embeddding dimension of GloVe word embeddings
    weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 

    """

    self.batch_size = batch_size
    self.output_size = output_size
    self.hidden_size = hidden_size
    self.vocab_size = vocab_size
    self.embedding_length = embedding_length

    self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
    self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False)
    self.rnn = nn.RNN(embedding_length, hidden_size, num_layers= 4, bidirection = True)
    self.label = nn.Linear(2 * 4 * hidden_size, output_size)
    self.act = nn.Softmax(output_size)
    self.dropout = nn.Dropout(p = 0.2)
    
	
  def forward(self, input_sentences, text_lengths, batch_size=None):
    input = self.word_embeddings(input_sentences)
    # input = nn.utils.rnn.pack_padded_sequence(input, text_lengths,batch_first=True)
    input = input.permute(1, 0, 2)
    if batch_size is None:
      h_0 = Variable(torch.zeros(4, self.batch_size, self.hidden_size).cuda()) # 4 = num_layers*num_directions
    else:
      h_0 =  Variable(torch.zeros(4, batch_size, self.hidden_size).cuda())
    output, h_n = self.rnn(input, h_0)
    # h_n.size() = (4, batch_size, hidden_size)
    h_n = self.dropout(h_n)
    h_n = h_n.permute(1, 0, 2) # h_n.size() = (batch_size, 4, hidden_size)
    h_n = h_n.contiguous().view(h_n.size()[0], h_n.size()[1]*h_n.size()[2])
    # h_n.size() = (batch_size, 4*hidden_size)
    logits = self.label(h_n) # logits.size() = (batch_size, output_size)

    return logits
    """ 
    Parameters
    ----------
    input_sentence: input_sentence of shape = (batch_size, num_sequences)
    batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
    
    Returns
    -------
    Output of the linear layer containing logits for pos & neg class which receives its input as the final_hidden_state of RNN.
    logits.size() = (batch_size, output_size)
    
    """
    

In [0]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    # optim = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
    # optim = torch.optim.SGD(model.parameters(), lr = 1e-5, momentum = 0.9, weight_decay = 0.005)
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text, text_lengths = batch.text
        target = batch.label
        # print(target, "target")
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
          text = text.cuda()
          target = target.cuda()
        if (text.size()[0] is not 32):
          # print("Here")
          # One of the batch returned by BucketIterator has length different than 32.
          continue
        optim.zero_grad()
        prediction = model(text, text_lengths).squeeze()
        # print(prediction.size())
        loss = loss_fn(prediction, target)
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        # print(target.data)
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        # if steps % 100 == 0:
            # print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text, text_lengths = batch.text
            if (text.size()[0] is not 32):
                continue
            target = batch.label
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            prediction = model(text, text_lengths)
            loss = loss_fn(prediction, target)
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)

def evaluate(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    y_test = []
    pred_test = []

    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text, text_lengths = batch.text
            if (text.size()[0] is not 32):
                continue
            target = batch.label
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            prediction = model(text, text_lengths)
            loss = loss_fn(prediction, target)
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
            y_test.append(target.data)
            pred_test.append(torch.max(prediction, 1)[1].view(target.size()).data)

            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter), y_test, pred_test





In [15]:
learning_rate = 2e-5
batch_size = 32
output_size = 13
# hidden_size = 128
hidden_size = 8
embedding_length = 300

model = RNN(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
loss_fn = F.cross_entropy

val_losses = []
train_losses = []
val_accs = []
train_accs = []
best_val_acc = 0
for epoch in range(100):
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
    if val_acc > best_val_acc:
      best_val_acc = val_acc
      torch.save(model.state_dict(), 'RNN_best2.pt')
      torch.save(model, 'RNN_best2')

TypeError: ignored

In [0]:
import matplotlib.pyplot as plt

epoch_list = []
for i in range(13):
  epoch_list.append(i + 1)
plt.plot(epoch_list, train_losses[:13])
# plt.plot(epoch_list, train_accs[:13])
# plt.plot(epoch_list, val_accs[:13])
plt.plot(epoch_list, val_losses[:13], '--r')
plt.legend(["train loss", "validation loss"])
# plt.legend(["train acc", "validation acc"])
plt.xlabel("epochs")
# plt.savefig("model1.jpeg", dpi = 500, format = "jpeg")
plt.show()




In [0]:
model = torch.load('RNN_best')
valid_loss, valid_acc, y_test, pred_test = evaluate(model, valid_iter)
import numpy as np
print(f'Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc:.2f}%')
def return_labels(tensor_list):
  new_list = np.array(tensor_list)
  labels = []
  for i in range(new_list.shape[0]):
    labels.append(new_list[i].squeeze().tolist())
  final_labels = []
  for i in range(len(labels)):
    for j in range(len(labels[i])):
      final_labels.append(labels[i][j])
  return final_labels

y_test = return_labels(y_test)
pred_test = return_labels(pred_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred_test)

import seaborn as sns
%matplotlib inline
sns.heatmap(cm, annot=True, cbar=True, cmap = 'YlGnBu', fmt = 'd')
plt.xlabel('True Label')
plt.ylabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()
# plt.savefig("cm_lstm_3.jpeg", dpi = 500, format = 'jpeg')